In [5]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim


In [6]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, ))
])
train_dataset = datasets.MNIST(root='./dataset/mnist/',
                train=True,
                download=False,
                transform=transform)
train_loader = DataLoader(train_dataset,
                shuffle=True,
                batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist/',
                train=False,
                download=False,
                transform=transform)
test_loader = DataLoader(test_dataset,
                shuffle=False,
                batch_size=batch_size)

In [7]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320, 10)
    def forward(self, x):
        # Flatten data from (n, 1, 28, 28) to (n, 784)
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1) # flatten
        x = self.fc(x)
        return x
model = Net()

In [8]:
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

True


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [11]:
# def train(epoch):
#     running_loss = 0.0
#     for batch_idx, data in enumerate(train_loader, 0):
#         inputs, target = data
#         optimizer.zero_grad()
#         # forward + backward + update
#         outputs = model(inputs)
#         loss = criterion(outputs, target)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         if batch_idx % 300 == 299:
#             print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
#             running_loss = 0.0

def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.to(device)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 2000))
            running_loss = 0.0

In [16]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            print('Accuracy on test set: %d %%' % (100 * correct / total))

In [17]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.011
[1,   600] loss: 0.011
[1,   900] loss: 0.010
Accuracy on test set: 100 %
Accuracy on test set: 100 %
Accuracy on test set: 100 %
Accuracy on test set: 99 %
Accuracy on test set: 99 %
Accuracy on test set: 98 %
Accuracy on test set: 98 %
Accuracy on test set: 98 %
Accuracy on test set: 98 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy on test set: 97 %
Accuracy o